<a href="https://colab.research.google.com/github/XingxinHE/Python-for-Data-Analysis-Notes/blob/master/7_2_Data_Transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

## Removing Duplicates

In [2]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
          'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [3]:
data.duplicated() #这个是看每一row有没有duplicate的，
#符合条件是以row为基准，例如最后那一row，只和row 5一样

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [4]:
data.drop_duplicates() #把duplicate的row给drop掉

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


以上这些方法都是默认考虑所有columns的，当然你可以specify which column

In [5]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [6]:
data.drop_duplicates(['k1']) #如果只考虑k1的话，那么从第三行开始就被删除了

,k1,k2,v1
0,one,1,0
1,two,1,1


In [7]:
data.drop_duplicates(['k1', 'k2'], keep='last')
#当specify两个column的时候，要满足两个column都是duplicate才可以drop
#keep=last是从下开始检索，保留最后面的，删除前面和它重复的

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


## Transforming Data Using a Function or Mapping
这个专题很重要，常用到

In [8]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
           'Pastrami', 'corned beef', 'Bacon',
           'pastrami', 'honey ham', 'nova lox'],
           'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


我们需要把每种食物的名字转换成肉类，比如salami是猪肉

In [9]:
meat_to_animal = {
    'bacon': 'pig',
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmon'
}

这种经典的map function需要的是一个dict

当这里有个问题，前面df某些value的大小写不一致，如bacon和Bacon，这在data cleaning里面经常碰到，需要做以下操作

In [10]:
lowercased = data['food'].str.lower() #将food column转换成小写格式
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [11]:
data['animal'] = lowercased.map(meat_to_animal) #利用map function，把肉类map出来
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


我们可以把上面的几个步骤汇聚成一个，很重要！很关键！

In [13]:
data['food'].map(lambda x: meat_to_animal[x.lower()])
#解读上面这段代码：
#对data['food']进行map function， map(function)
#这个function是什么？
#是一个匿名函数lambda x
#lambda x 是把 x.lower()小写的x，也就是data['food']的value 写入meat_to_animal这个dict
#因为写入的是key,所以return的是meat_to_animal的value，即肉类

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

🌟**Using map is a convenient way to perform element-wise transformations and other data cleaning-related operations!!!!!!!!**

## Replacing Values
.replace()

In [14]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [16]:
data.replace(-999, np.nan) #经常会用到replace，例如这个series的-999明显是outlier，所以要replace掉

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [17]:
data.replace([-999, -1000], np.nan) #replace多个value

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [18]:
data.replace([-999, -1000], [np.nan, 0]) #2对2的replace，前提是被replace和replace物的shape一致

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [19]:
data.replace({-999: np.nan, -1000: 0}) #这种用dict的方法最稳妥

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

## Renaming Axis Indexes

In [20]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
          index=['Ohio', 'Colorado', 'New York'],
          columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


一种方法是设定一个function，然后map这个function来修改名字

In [21]:
transform = lambda x: x[:4].upper() #取前4个字节，然后大写它们
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [23]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


上面的方法比较累赘，如果只是modify标题的大小写的话，因为pandas为此设计了更方便的function

.rename()

In [24]:
data.rename(index=str.title, columns=str.upper) #这真的强，一行代码解决了row和column的rename

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [25]:
#同理，rename也可以接受一个dict
data.rename(index={'OHIO': 'INDIANA'},
      columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [26]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
#inplace就是直接修改原df

In [27]:
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


## Discretization and Binning

Continuous data is often discretized or otherwise separated into "bins" for analysis. Suppose you have data about a group of people in a study, and you want to group them into discrete age buckets

连续的数据通常被离散化或以其他方式分离成 "bin"进行分析。假设在一项研究中，你有一群人的数据，你想把他们分成离散的年龄段。

说大白话点，就是把一个continuous的data分成段(bin)

pd.cut()

In [28]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32] #离散化的数字
bins = [18, 25, 35, 60, 100] #bin的domain

In [29]:
cuts = pd.cut(ages, bins)
cuts #可以看到各数值分布在哪些domain之间

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

output是一个特殊的categories的object type

In [30]:
cuts.codes #看看具体在哪个分区，比如第一个(18,25]是第一个分区，所以为0

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [32]:
cuts.categories #这个则是还原在哪个分区

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [33]:
pd.value_counts(cuts) #这个可以把codes&categories统合在一起，看各分区都有多少个

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

( means open,  [ means inclusive，这个可以change的

In [34]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

接下来就牛逼了，我们把年龄段分出来之后，肯定会把这些年龄段命名嘛，如下

In [35]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

前面提到的是pd.cut(值，划分的区间）

还有一种方法是pd.cut（值，多少个区间），这种方法类似于grasshopper的divide domain

In [36]:
data = np.random.rand(20)
data

array([0.33116313, 0.4816452 , 0.39264393, 0.68744012, 0.31096172,
       0.66502406, 0.48679187, 0.46737955, 0.34916652, 0.80639632,
       0.5535195 , 0.04281284, 0.29324093, 0.44839778, 0.995994  ,
       0.935461  , 0.88781025, 0.93660602, 0.17922879, 0.7414792 ])

In [37]:
pd.cut(data, 4, precision=2) #把这些值统计出来bounds是多少（GH的思想），然后把这个bounds得出来的domain / 4
#precision是精确到最后几位数

[(0.28, 0.52], (0.28, 0.52], (0.28, 0.52], (0.52, 0.76], (0.28, 0.52], ..., (0.76, 1.0], (0.76, 1.0], (0.76, 1.0], (0.042, 0.28], (0.52, 0.76]]
Length: 20
Categories (4, interval[float64]): [(0.042, 0.28] < (0.28, 0.52] < (0.52, 0.76] < (0.76, 1.0]]

上文提及的是divide domain的方法， 下面则是根据输入value的数目进行等分，类似于GH的partition list

pd.qcut()                       
q是quantiles的意思




In [40]:
data = np.random.randn(1000) # Normally distributed
cuts = pd.qcut(data, 4) # Cut into quartiles

In [41]:
cuts

[(-0.744, -0.0376], (-0.0376, 0.601], (0.601, 2.905], (0.601, 2.905], (0.601, 2.905], ..., (-3.8489999999999998, -0.744], (-3.8489999999999998, -0.744], (-0.744, -0.0376], (0.601, 2.905], (0.601, 2.905]]
Length: 1000
Categories (4, interval[float64]): [(-3.8489999999999998, -0.744] < (-0.744, -0.0376] < (-0.0376, 0.601] <
                                    (0.601, 2.905]]

In [42]:
pd.value_counts(cuts)
#可以看到，它自动取值了4个区间，让每个区间都是250个item

(0.601, 2.905]                   250
(-0.0376, 0.601]                 250
(-0.744, -0.0376]                250
(-3.8489999999999998, -0.744]    250
dtype: int64

## Detecting and Filtering Outliers